In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=4251ea94dd9d0547aced51ec9553e6d0d271f12fc2bbf714f72616d074a954fb
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-04 23:30:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2023-04-04 23:30:04 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(25)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...|2013-02-13 00:00:00|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(25)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      26425|             1|
|      29814|             1|
|      78440|             2|
|      90550|             1|
|      99817|             1|
|    2846240|             1|
|    2966808|             1|
|    3323007|             1|
|    3458442|             1|
|    3497479|             1|
|    3498767|             1|
|    3511947|             1|
|    3945593|             1|
|    4049807|             1|
|    4214333|             1|
|    4262489|             2|
|    4423902|             1|
|    4660180|             2|
|    4667529|             3|
|    4838481|             2|
|    4841869|             1|
|    5023675|             6|
|    5030898|             1|
|    5101867|             5|
|    5201254|             1|
+-----------+--------------+
only showing top 25 rows



In [15]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = df.select(['product_id','product_title']).drop_duplicates()


product_id  product_title                                                                                                                                        
0001047663  A Matter of Honor                                                                                                                                        1
B0054N73EY  I'm with You                                                                                                                                             1
B0054JZC6E  101-in-1 Games                                                                                                                                           1
B0054KCGCG  The Experience Economy, Updated Edition                                                                                                                  1
B0054KRCDE  At Fillmore East (Deluxe Edition)                                                                                                                        1
   

In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(25)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R32VYUWDIB5LKE|      14952|0552774294|     362925721| 2013-02-13|
|R3CCMP4EV6HAVL|      14952|B004GJXQ20|     268067011| 2014-08-03|
|R14NAE6UGTVTA2|      17564|B00GIGGS6A|     256731097| 2015-07-07|
|R2E7QEWSC6EWFA|      18940|B00CW7KK9K|     977480037| 2014-06-16|
|R26E6I47GQRYKR|      20315|B002L6SKIK|     827187473| 2013-06-10|
|R1RJMTSNCKB9LP|      20842|B00004YLIU|     678427290| 2013-04-15|
|R2P2XF84YELQBZ|      20913|B00AYHK7RU|     108403123| 2013-06-07|
| RJKULSX2Y5R07|      21490|B00CJ3V5UK|     252503117| 2014-11-07|
|R3UYE0U7SQCI8Q|      24196|B000FUM0TE|      48021829| 2012-11-16|
|R1TKJ7XFS3RDEB|      24196|B000UMXCTY|     181554537| 2013-04-23|
|R3S9JNS21QDBXP|      24196|B0044JV1K6|     386772628| 2013-08-02|
|R2Y4O06QMOGHD0|      24196|2226249672|     753674225| 2013-10

In [9]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(25)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32VYUWDIB5LKE|          5|            0|          0|   N|                Y|
|R3CCMP4EV6HAVL|          5|            0|          0|   N|                Y|
|R14NAE6UGTVTA2|          3|            1|          3|   N|                Y|
|R2E7QEWSC6EWFA|          4|            0|          1|   N|                Y|
|R26E6I47GQRYKR|          2|            3|          5|   N|                N|
|R1RJMTSNCKB9LP|          2|            0|          0|   N|                Y|
|R2P2XF84YELQBZ|          5|            0|          3|   N|                Y|
| RJKULSX2Y5R07|          5|            0|          0|   N|                Y|
|R3UYE0U7SQCI8Q|          5|            2|          3|   N|                Y|
|R1TKJ7XFS3RDEB|          5|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://heimpel-mod17.c3qyrkygwvwo.us-east-2.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": "Matt&Ben0813", 
          "driver":"org.postgresql.Driver"}

In [47]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [44]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [48]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)